In [ ]:
trf_fts = fits.open("./gof_hri_174_sun_coronal_2021_chianti.abund_chianti.ioneq_synthetic.fits")

trf = trf_fts[0].data
# hd = trf_fts[0].header
sv = np.array(trf[1] * 1e-8, dtype=np.float64)
#norm [-30,-23] to [0,1]
sv = np.log10(sv)
sv = (sv+30)/(30-23)

# print(sv.shape)
# print(type(sv[0]))

np.save('./TRF/174.npy', sv)



In [ ]:

import os
import numpy as np
import astropy.io.fits as fits
# from astropy.visualization import ImageNormalize, AsinhStretch
from glob import glob
from tqdm import tqdm

hri_list = glob('./174_L2/*.fits')
data_num = len(hri_list)

print(data_num)

for i in tqdm(range(data_num)):

    img = fits.open(hri_list[i])
    dat = img[1].data
    hdr = img[1].header
    
    data_name = hri_list[i].split('/')[-1].split('.')[0]
    
    dat[dat<0] = 0
    data = np.nan_to_num(dat)

    dsun = hdr["DSUN_OBS"]  # 태양과의 거리 (단위: m)
    au = 1.496e11  # 1 AU (m)
    
    correction_factor = (dsun / au)**2  # Conrrection factor to 1 AU
    data_corrected = data * correction_factor

    UpLim = 15
    LoLim = 0

    log2_data = np.log2(data_corrected + 1) # log2
    log2_data = (np.clip(log2_data, LoLim, UpLim) - (UpLim-LoLim))/((UpLim-LoLim)) #normalize to 0~1
    log2_data = np.float32(log2_data)   # float32

    np.save(f'./HRI/174_L2_npy/174/{data_name}.npy', log2_data)
    if i % 100 == 0:
        print(data_name, f"{i}/{data_num}")
